# Oscar Science Using Random Forest Classifier

Using Dataset provided by Thinkful for winners and losers over the past 38 years, this approach made correct predictions in all but 1 year, 2017.

### Imports and loading data

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

pd.set_option('mode.chained_assignment', None)

In [2]:
train_file = "train.csv"
initial_train = pd.read_csv(train_file)

train = initial_train[(initial_train['Year'] > 1980)]

test_file = "test.csv"
test = pd.read_csv(test_file)

Data

You need to "run" the two cells below, to do that select the cell and press:  *`Shift-Enter`*

In [3]:
train.head(5)

,Year,Movie,Won?,Budget,Opening Weekend,IMDB Rating,Genres,Won Golden Globe,Won Bafta,Oscar Nominations,Golden Globe Nominations,Bafta Nominations,IMdB id,Won Producers,Won Directors,Won Actors,Rate,Metascore
0,2016,Arrival,0,47000000.0,24000000.0,8.1,"Drama, Mystery, Sci-Fi, Thriller",0,0,8,2,9,tt2543164,0,0,0,PG-13,81.0
1,2016,Fences,0,24000000.0,129462.0,7.5,Drama,0,0,4,2,1,tt2671706,0,0,0,PG-13,79.0
2,2016,Hacksaw Ridge,0,40000000.0,15190758.0,8.3,"Drama, History, War",0,0,6,3,5,tt2119532,0,0,0,R,71.0
3,2016,Hell or High Water,0,12000000.0,621329.0,7.7,"Action, Crime, Drama, Western",0,0,4,3,3,tt2582782,0,0,0,R,88.0
4,2016,Hidden Figures,0,25000000.0,515499.0,7.9,"Biography, Drama, History",0,0,3,2,1,tt4846340,0,0,1,PG,74.0


In [4]:
train['Won?'].value_counts()

0    159
1     33
Name: Won?, dtype: int64

## Formatting

In [5]:
# Set Rate to a number to be able to analyze it
train.ix[train["Rate"] == "G", "Rate"] = 1
train.ix[train["Rate"] == "PG", "Rate"] = 2
train.ix[train["Rate"] == "PG-13", "Rate"] = 3
train.ix[train["Rate"] == "R", "Rate"] = 4

test.ix[test["Rate"] == "G", "Rate"] = 1
test.ix[test["Rate"] == "PG", "Rate"] = 2
test.ix[test["Rate"] == "PG-13", "Rate"] = 3
test.ix[test["Rate"] == "R", "Rate"] = 4

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [6]:
train.head(5)

,Year,Movie,Won?,Budget,Opening Weekend,IMDB Rating,Genres,Won Golden Globe,Won Bafta,Oscar Nominations,Golden Globe Nominations,Bafta Nominations,IMdB id,Won Producers,Won Directors,Won Actors,Rate,Metascore
0,2016,Arrival,0,47000000.0,24000000.0,8.1,"Drama, Mystery, Sci-Fi, Thriller",0,0,8,2,9,tt2543164,0,0,0,3,81.0
1,2016,Fences,0,24000000.0,129462.0,7.5,Drama,0,0,4,2,1,tt2671706,0,0,0,3,79.0
2,2016,Hacksaw Ridge,0,40000000.0,15190758.0,8.3,"Drama, History, War",0,0,6,3,5,tt2119532,0,0,0,4,71.0
3,2016,Hell or High Water,0,12000000.0,621329.0,7.7,"Action, Crime, Drama, Western",0,0,4,3,3,tt2582782,0,0,0,4,88.0
4,2016,Hidden Figures,0,25000000.0,515499.0,7.9,"Biography, Drama, History",0,0,3,2,1,tt4846340,0,0,1,2,74.0


## Cleaning

In [7]:
train["IMDB Rating"].fillna(train["IMDB Rating"].median(), inplace=True)
test["IMDB Rating"].fillna(test["IMDB Rating"].median(), inplace=True)

train["Metascore"].fillna(train["Metascore"].median(), inplace=True)
test["Metascore"].fillna(train["Metascore"].median(), inplace=True)

## Using Decision Tree

In [8]:
target = train["Won?"].values

feature_names = [
    "Oscar Nominations",
    "Won Golden Globe",
    "Golden Globe Nominations",
    "Won Bafta",
    "Bafta Nominations",
    "Won Producers",
    "Won Actors",
    "Won Directors",
    "Metascore",
    "IMDB Rating"]

features = train[feature_names].values

# Fit your first decision tree: my_tree
my_tree = tree.DecisionTreeClassifier()
my_tree = my_tree.fit(features, target)

In [9]:
tree_importances = pd.DataFrame(my_tree.feature_importances_, feature_names, columns=["Importances"])

print(tree_importances)
print('Score', my_tree.score(features, target))

                          Importances
Oscar Nominations            0.170111
Won Golden Globe             0.042691
Golden Globe Nominations     0.033543
Won Bafta                    0.002749
Bafta Nominations            0.126074
Won Producers                0.000000
Won Actors                   0.025921
Won Directors                0.402053
Metascore                    0.059212
IMDB Rating                  0.137644
Score 1.0


## Predicting

In [10]:
test_features = test[feature_names].values

pred_tree = my_tree.predict_proba(test_features)[:, 1]

movie_name = np.array(test["Movie"])
year = np.array(test["Year"])
won = np.array(test["Won?"])

tree_prediction = pd.DataFrame(pred_tree.round(2), movie_name, columns=["Probability"])
tree_prediction["Year"] = year
tree_prediction["Actually Won?"] = won

In [11]:
tree_prediction[tree_prediction['Year'] != 2017]

,Probability,Year,Actually Won?
Avatar,0.0,2009,0.0
The Blind Side,0.0,2009,0.0
District 9,0.0,2009,0.0
An Education,0.0,2009,0.0
The Hurt Locker,0.0,2009,1.0
Inglourious Basterds,1.0,2009,0.0
Precious: Based on the Novel 'Push' by Sapphire,0.0,2009,0.0
A Serious Man,0.0,2009,0.0
Up,0.0,2009,0.0
Up in the Air,0.0,2009,0.0


## Overfitting

* Resulting model too tied to the training set.
* It doesn’t generalize to new data, which is the point of prediction.

## Random Forest Classifier

Random Forest Classifiers use many decision trees to build a classifier.  The data is randomly subset, a decision tree is built, and this process is repeated many times (1000 times in our case).  Finally the information that is gained through the many decision trees is used to create the random forest classifier

In [12]:
forest = RandomForestClassifier(
    max_depth=25,
    min_samples_split=15,
    n_estimators=1000,
    random_state=1)

my_forest = forest.fit(features, target)

In [13]:
forest_importances = pd.DataFrame(my_forest.feature_importances_, feature_names, columns=["Importances"])

print(forest_importances)
print('Score', my_forest.score(features, target))

                          Importances
Oscar Nominations            0.116006
Won Golden Globe             0.053063
Golden Globe Nominations     0.045208
Won Bafta                    0.018955
Bafta Nominations            0.065676
Won Producers                0.130809
Won Actors                   0.076590
Won Directors                0.357308
Metascore                    0.060241
IMDB Rating                  0.076142
Score 0.9375


## Predicting with Random Forest Classifier

In [14]:
pred_forest = my_forest.predict_proba(test_features)[:, 1]

forest_prediction = pd.DataFrame(pred_forest, movie_name, columns=["Probability"])
forest_prediction["Year"] = year
forest_prediction["Actually Won?"] = won

In [15]:
normalized_prediction = forest_prediction.copy()

for index, row in normalized_prediction.iterrows():
    normalized_prediction.loc[index, "Probability"] = \
        (row["Probability"] / forest_prediction["Probability"][forest_prediction["Year"] == row["Year"]].sum()).round(2)

In [16]:
normalized_prediction[normalized_prediction["Year"] == 1976].sort_values("Probability", ascending=False)

,Probability,Year,Actually Won?
Rocky,0.75,1976,1.0
Network,0.12,1976,0.0
Taxi Driver,0.09,1976,0.0
All the President's Men,0.04,1976,0.0
Bound for Glory,0.00,1976,0.0


In [17]:
normalized_prediction[normalized_prediction["Year"] == 1984].sort_values("Probability", ascending=False)

,Probability,Year,Actually Won?
Amadeus,0.74,1984,1.0
A Passage to India,0.13,1984,0.0
The Killing Fields,0.11,1984,0.0
A Soldier's Story,0.02,1984,0.0
Places in the Heart,0.01,1984,0.0


In [18]:
normalized_prediction[normalized_prediction["Year"] == 1996].sort_values("Probability", ascending=False)

,Probability,Year,Actually Won?
The English Patient,0.92,1996,1.0
Fargo,0.05,1996,0.0
Secrets & Lies,0.02,1996,0.0
Shine,0.01,1996,0.0
Jerry Maguire,0.00,1996,0.0


In [19]:
normalized_prediction[normalized_prediction["Year"] == 2009].sort_values("Probability", ascending=False)

,Probability,Year,Actually Won?
The Hurt Locker,0.40,2009,1.0
Inglourious Basterds,0.28,2009,0.0
Up,0.15,2009,0.0
Avatar,0.08,2009,0.0
Up in the Air,0.03,2009,0.0
District 9,0.02,2009,0.0
The Blind Side,0.01,2009,0.0
An Education,0.01,2009,0.0
A Serious Man,0.01,2009,0.0
Precious: Based on the Novel 'Push' by Sapphire,0.00,2009,0.0


In [20]:
normalized_prediction[normalized_prediction["Year"] == 2017].sort_values("Probability", ascending=False)

,Probability,Year,Actually Won?
The Shape of Water,0.48,2017,NaN
"Three Billboards Outside Ebbing, Missouri",0.28,2017,NaN
Dunkirk,0.07,2017,NaN
Call Me By Your Name,0.06,2017,NaN
Lady Bird,0.04,2017,NaN
Phantom Thread,0.03,2017,NaN
Darkest Hour,0.02,2017,NaN
The Post,0.02,2017,NaN
Get Out,0.00,2017,NaN
